In [40]:
import os
from dotenv import load_dotenv
import requests
import json
from datetime import datetime, timedelta
import pandas as pd
import time

load_dotenv()

True

In [41]:
#vehicle count data
url = "https://data.ny.gov/resource/wujg-7c2s.json"
limit = 100000#1000000
max_rows = 1000000 #110696370
offset = 0
metro_df = pd.DataFrame()
nyc_token = os.getenv("TRAFFIC_KEY")

In [42]:
while offset < max_rows:
    this_url = url + f"?$limit={limit}&$offset={offset}&$order=:id&$$app_token={nyc_token}"
    print(this_url)
    response = requests.get(this_url)
    if response.status_code == 200:
        json_data = response.json()
        temp_df = pd.DataFrame(json_data)

        print(f"Temp Length {len(temp_df)}")

        #drop extra cols
        temp_df = temp_df[['transit_timestamp', 'transit_mode', 'borough', 'payment_method', 'ridership']]


        #filter only manhatan
        temp_df = temp_df[temp_df['borough'].str.lower() == 'manhattan']

        #filter out before 2020
        temp_df['transit_timestamp'] = pd.to_datetime(temp_df['transit_timestamp'])
        temp_df = temp_df[temp_df['transit_timestamp'] >= '2020-01-01']

        print(f"Temp Length after cutoff {len(temp_df)}")
        metro_df = pd.concat([metro_df, temp_df])
        
    else:
        print(f"Error: {response.status_code}")
    offset = offset + limit
    print(f"Total len {len(metro_df)}")
    time.sleep(1)


https://data.ny.gov/resource/wujg-7c2s.json?$limit=100000&$offset=0&$order=:id&$$app_token=wwBwMlUXanS8k91CYV0vJ3NlT
Temp Length 100000
Temp Length after cutoff 30444
Total len 30444
https://data.ny.gov/resource/wujg-7c2s.json?$limit=100000&$offset=100000&$order=:id&$$app_token=wwBwMlUXanS8k91CYV0vJ3NlT
Temp Length 100000
Temp Length after cutoff 30504
Total len 60948
https://data.ny.gov/resource/wujg-7c2s.json?$limit=100000&$offset=200000&$order=:id&$$app_token=wwBwMlUXanS8k91CYV0vJ3NlT
Temp Length 100000
Temp Length after cutoff 30708
Total len 91656
https://data.ny.gov/resource/wujg-7c2s.json?$limit=100000&$offset=300000&$order=:id&$$app_token=wwBwMlUXanS8k91CYV0vJ3NlT
Temp Length 100000
Temp Length after cutoff 30522
Total len 122178
https://data.ny.gov/resource/wujg-7c2s.json?$limit=100000&$offset=400000&$order=:id&$$app_token=wwBwMlUXanS8k91CYV0vJ3NlT
Temp Length 100000
Temp Length after cutoff 30233
Total len 152411
https://data.ny.gov/resource/wujg-7c2s.json?$limit=100000&$offs

In [43]:
len(metro_df)

305013

In [44]:
metro_df.head()

,transit_timestamp,transit_mode,borough,payment_method,ridership
5,2024-05-17 19:00:00,subway,Manhattan,omny,4.0
10,2024-05-17 05:00:00,subway,Manhattan,metrocard,8.0
11,2024-05-17 12:00:00,subway,Manhattan,metrocard,43.0
12,2024-05-17 05:00:00,subway,Manhattan,metrocard,3.0
13,2024-05-17 08:00:00,subway,Manhattan,metrocard,10.0


In [45]:
metro_df['date'] = metro_df['transit_timestamp'].dt.date
metro_df['time'] = metro_df['transit_timestamp'].dt.time

In [46]:
metro_df.head()

,transit_timestamp,transit_mode,borough,payment_method,ridership,date,time
5,2024-05-17 19:00:00,subway,Manhattan,omny,4.0,2024-05-17,19:00:00
10,2024-05-17 05:00:00,subway,Manhattan,metrocard,8.0,2024-05-17,05:00:00
11,2024-05-17 12:00:00,subway,Manhattan,metrocard,43.0,2024-05-17,12:00:00
12,2024-05-17 05:00:00,subway,Manhattan,metrocard,3.0,2024-05-17,05:00:00
13,2024-05-17 08:00:00,subway,Manhattan,metrocard,10.0,2024-05-17,08:00:00


In [47]:
metro_df['ridership'] = metro_df['ridership'].astype(float)


In [48]:
metro_df.groupby('date', as_index=False)['ridership'].sum()


,date,ridership
0,2024-03-13,2283179.0
1,2024-04-05,2121297.0
2,2024-04-16,290039.0
3,2024-04-19,2037953.0
4,2024-05-01,2295041.0
5,2024-05-02,2308461.0
6,2024-05-03,2088576.0
7,2024-05-07,2331842.0
8,2024-05-08,2337802.0
9,2024-05-09,2335541.0
